In [19]:
from SDP_interaction_inference.constraints import Constraint
from SDP_interaction_inference import optimization
from SDP_interaction_inference.dataset import Dataset
from SDP_interaction_inference.correlation import Correlation
from SDP_interaction_inference import utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import scipy
import tqdm
from copy import deepcopy
import matplotlib.patches
import corner
import plotly
import plotly.graph_objects as go

# Model Free Investigation

## Data Loading

In [20]:
# read fibroblast transcript counts
data_FIB = pd.read_csv("./Data/GSE151334_FIB_counts_thresh.csv", index_col=0)

# load capture
beta = np.loadtxt("./Capture/beta_FIB.txt")

# load RNA types
biotypes_dict = json.load(open("./Biotypes/biotypes_FIB.json"))

# select indices of protein coding mRNA and non-coding miRNA
pcRNA_indices = [idx for idx, btype in enumerate(biotypes_dict.values()) if btype == "protein_coding"]
miRNA_indices = [idx for idx, btype in enumerate(biotypes_dict.values()) if btype == "miRNA"]

# separate data
data_FIB_pcRNA = data_FIB.iloc[pcRNA_indices]
data_FIB_miRNA = data_FIB.iloc[miRNA_indices]

# amounts: see './Biotypes/updated-biotypes.ipynb' for full pie chart of types
print(f"Total of {data_FIB.shape[0]} genes with mean > 1 present \nSelected {data_FIB_miRNA.shape[0]} non-coding miRNA \nSelected {data_FIB_pcRNA.shape[0]} protein coding (mRNA)")

Total of 11747 genes with mean > 1 present 
Selected 19 non-coding miRNA 
Selected 9296 protein coding (mRNA)


## Setup functions

In [21]:
def construct_dataset(mirna_sample, mrna_dataset, beta, resamples=1000):

    # size
    gene_pairs, cells = mrna_dataset.shape

    # construct paired count dataframe
    counts_df = pd.DataFrame(
        index = [f"Gene-pair-{i}" for i in range(gene_pairs)],
        columns = [f"Cell-{j}" for j in range(cells)]
    )

    # fill with pairs
    for i in range(gene_pairs):
        gene_i = mirna_sample
        gene_j = mrna_dataset.iloc[i]
        gene_pair_ij = list(zip(gene_i, gene_j))
        counts_df.iloc[i] = gene_pair_ij

    # construct dataset object
    data = Dataset()
    data.count_dataset = counts_df
    data.cells = cells
    data.gene_pairs = gene_pairs

    # settings
    data.resamples = resamples

    # set capture
    data.beta = beta

    return data

In [22]:
def model_free_independence_test(dataset, d, **kwargs):

    # Independent model free
    constraints = Constraint(
        moment_bounds=True,
        moment_matrices=True,
        factorization=True
    )
    opt_MF_ind = optimization.ModelFreeOptimization(dataset, d, constraints=constraints, printing=False, silent=True, **kwargs)
    opt_MF_ind.analyse_dataset()

    # extract results
    status = [solution['status'] for solution in opt_MF_ind.result_dict.values()]
    times  = [solution['time'] for solution in opt_MF_ind.result_dict.values()]
    cuts   = [solution['cuts'] for solution in opt_MF_ind.result_dict.values()]

    # collect
    data = {
        'status': status,
        'times': times,
        'cuts': cuts,
        'model': opt_MF_ind
    }

    return data

In [23]:
def model_free_interacting_test(dataset, d, **kwargs):

    # Interacting model free
    constraints = Constraint(
        moment_bounds=True,
        moment_matrices=True,
        factorization=False
    )
    opt_MF_int = optimization.ModelFreeOptimization(dataset, d, constraints=constraints, printing=False, silent=True, **kwargs)
    opt_MF_int.analyse_dataset()

    # extract results
    status = [solution['status'] for solution in opt_MF_int.result_dict.values()]
    times  = [solution['time'] for solution in opt_MF_int.result_dict.values()]
    cuts   = [solution['cuts'] for solution in opt_MF_int.result_dict.values()]
    correlation = [solution['correlation'] for solution in opt_MF_int.result_dict.values()]

    # collect
    data = {
        'status': status,
        'times': times,
        'cuts': cuts,
        'correlation': correlation,
        'model': opt_MF_int
    }

    return data

## Setup results

### Construct

In [24]:
# select mRNA
mRNA = data_FIB_pcRNA.index

In [ ]:
# dataframe to store model free results
#MF_independent_df = pd.DataFrame(
#    index=mRNA
#)
#MF_interacting_df = pd.DataFrame(
#    index=mRNA
#)

### Load

In [ ]:
#MF_independent_df = pd.read_csv("Results/independent_MF.csv", index_col=0)
#MF_interacting_df = pd.read_csv("Results/interacting_MF.csv", index_col=0)

### Save

In [ ]:
#MF_independent_df.to_csv("Results/independent_MF.csv")
#MF_interacting_df.to_csv("Results/interacting_MF.csv")

## Running

In [ ]:
# d = 2


Running MIR100
Bootstrap:


100%|██████████| 9296/9296 [02:31<00:00, 61.19it/s]


MF Independence test:


100%|██████████| 9296/9296 [02:55<00:00, 53.05it/s]


(array(['INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([ 357, 8939]))
MF Interacting test:


...

MF Independence test:


100%|██████████| 9296/9296 [02:56<00:00, 52.74it/s]


(array(['INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([  35, 9261]))
MF Interacting test:


100%|██████████| 9296/9296 [02:39<00:00, 58.17it/s]


(array(['OPTIMAL'], dtype='<U7'), array([9296]))


In [ ]:
# for each miRNA
miRNA_names = data_FIB_miRNA.index
for miRNA in miRNA_names:

    # display miRNA
    print(f"\nRunning {miRNA}")

    # construct dataset of miRNA paired with mRNA
    dataset_SDP = construct_dataset(data_FIB_miRNA.loc[miRNA], data_FIB_pcRNA, beta)

    # bootstrap
    print("Bootstrap:")
    dataset_SDP.confidence = 0.95
    dataset_SDP.bootstrap(d=3, tqdm_disable=False)

    # independence test
    print("MF Independence test:")
    MF_ind_d3_c95 = model_free_independence_test(dataset_SDP, d=3, time_limit=30, total_time_limit=30)

    # display results
    print(np.unique(MF_ind_d3_c95['status'], return_counts=True))

    # interacting test
    print("MF Interacting test:")
    MF_int_d3_c95 = model_free_interacting_test(dataset_SDP, d=3, time_limit=30, total_time_limit=30)

    # display results
    print(np.unique(MF_int_d3_c95['status'], return_counts=True))

    # store results on dataframe

    # independence test
    MF_independent_df[f'{miRNA}_d3_c95_status'] = MF_ind_d3_c95['status']
    MF_independent_df[f'{miRNA}_d3_c95_times'] = MF_ind_d3_c95['times']
    MF_independent_df[f'{miRNA}_d3_c95_cuts'] = MF_ind_d3_c95['cuts']

    # interacting test
    MF_interacting_df[f'{miRNA}_d3_c95_status'] = MF_int_d3_c95['status']
    MF_interacting_df[f'{miRNA}_d3_c95_times'] = MF_int_d3_c95['times']
    MF_interacting_df[f'{miRNA}_d3_c95_cuts'] = MF_int_d3_c95['cuts']
    MF_interacting_df[f'{miRNA}_d3_c95_correlation'] = MF_int_d3_c95['correlation']

    # save dataframes as safety
    #MF_independent_df.to_csv("Results/independent_MF_d3.csv")  # Uncomment when running
    #MF_interacting_df.to_csv("Results/interacting_MF_d3.csv")


Running MIR100
Bootstrap:


100%|██████████| 9296/9296 [03:55<00:00, 39.41it/s]


MF Independence test:


100%|██████████| 9296/9296 [05:07<00:00, 30.19it/s]


(array(['INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([1037, 8259]))
MF Interacting test:


...

MF Independence test:


100%|██████████| 9296/9296 [05:03<00:00, 30.67it/s]


(array(['INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([ 484, 8812]))
MF Interacting test:


100%|██████████| 9296/9296 [04:44<00:00, 32.64it/s]


(array(['INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([   1, 9295]))


In [ ]:
# d = 4


Running MIR100
Bootstrap:


100%|██████████| 9296/9296 [05:51<00:00, 26.43it/s]


MF Independence test:


100%|██████████| 9296/9296 [08:49<00:00, 17.55it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([  12, 1916, 7368]))
MF Interacting test:


...

MF Independence test:


100%|██████████| 9296/9296 [08:45<00:00, 17.70it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([  15, 1662, 7619]))
MF Interacting test:


100%|██████████| 9296/9296 [10:24<00:00, 14.89it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([1597,   61, 7638]))


In [ ]:
# Old dataset: filtered genes first, and using bugged capture


Running MIR100
Bootstrap:


100%|██████████| 9376/9376 [05:57<00:00, 26.25it/s]


MF Independence test:


100%|██████████| 9376/9376 [08:40<00:00, 18.02it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([  10, 1926, 7440]))
MF Interacting test:


...

MF Independence test:


100%|██████████| 9376/9376 [10:15<00:00, 15.23it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([  13, 1696, 7667]))
MF Interacting test:


 71%|███████   | 6671/9376 [08:16<04:10, 10.78it/s]c:\Users\willi\Documents\ProjectPaper\M5R_venv\lib\site-packages\SDP_interaction_inference\optimization_utils.py:615: RuntimeWarning: invalid value encountered in sqrt
  correlation = cov_xy / (np.sqrt(var_x) * np.sqrt(var_y))
100%|██████████| 9376/9376 [11:38<00:00, 13.42it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([1579,   87, 7710]))


## Notes

- some NUMERIC status
- varying levels of dependence
- overall a lot of CUT_LIMIT for interacting test, and some INFEASIBLE

## Testing

### Reduce to order 3

In [9]:
# select miRNA
miRNA = "MIR103A2"

# display miRNA
print(f"\nRunning {miRNA}")

# construct dataset of miRNA paired with mRNA
dataset_SDP = construct_dataset(data_FIB_miRNA.loc[miRNA], data_FIB_pcRNA, beta)

# bootstrap
print("Bootstrap:")
dataset_SDP.confidence = 0.95
dataset_SDP.bootstrap(d=3, tqdm_disable=False)

# independence test
print("MF Independence test:")
MF_ind_d3_c95 = model_free_independence_test(dataset_SDP, d=3, time_limit=30, total_time_limit=30)

# display results
print(np.unique(MF_ind_d3_c95['status'], return_counts=True))

# interacting test
print("MF Interacting test:")
MF_int_d3_c95 = model_free_interacting_test(dataset_SDP, d=3, time_limit=30, total_time_limit=30)

# display results
print(np.unique(MF_int_d3_c95['status'], return_counts=True))


Running MIR103A2
Bootstrap:


100%|██████████| 9296/9296 [04:11<00:00, 36.94it/s]


MF Independence test:


100%|██████████| 9296/9296 [05:13<00:00, 29.69it/s]


(array(['INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([1677, 7619]))
MF Interacting test:


100%|██████████| 9296/9296 [04:46<00:00, 32.42it/s]

(array(['OPTIMAL'], dtype='<U7'), array([9296]))


### Reduce to order 2

In [10]:
# select miRNA
miRNA = "MIR103A2"

# display miRNA
print(f"\nRunning {miRNA}")

# construct dataset of miRNA paired with mRNA
dataset_SDP = construct_dataset(data_FIB_miRNA.loc[miRNA], data_FIB_pcRNA, beta)

# bootstrap
print("Bootstrap:")
dataset_SDP.confidence = 0.95
dataset_SDP.bootstrap(d=2, tqdm_disable=False)

# independence test
print("MF Independence test:")
MF_ind_d2_c95 = model_free_independence_test(dataset_SDP, d=2, time_limit=30, total_time_limit=30)

# display results
print(np.unique(MF_ind_d2_c95['status'], return_counts=True))

# interacting test
print("MF Interacting test:")
MF_int_d2_c95 = model_free_interacting_test(dataset_SDP, d=2, time_limit=30, total_time_limit=30)

# display results
print(np.unique(MF_int_d2_c95['status'], return_counts=True))


Running MIR103A2
Bootstrap:


100%|██████████| 9296/9296 [02:30<00:00, 61.69it/s]


MF Independence test:


100%|██████████| 9296/9296 [02:56<00:00, 52.68it/s]


(array(['INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([ 607, 8689]))
MF Interacting test:


100%|██████████| 9296/9296 [02:43<00:00, 56.86it/s]

(array(['OPTIMAL'], dtype='<U7'), array([9296]))
